# PARCELS Tutorial

Welcome to a quick tutorial on PARCELS. This is meant to get you started with the code, and give you a flavour of some of the key features of PARCELS.

In this tutorial, we will first cover how to run a set of particles [from a very simple idealised grid](#idealised). Then, we will show how to [add custom behaviour](#customkernel) to the particles. And finally, we will show how to [run particles in a set of NetCDF files](#netcdf)

In [ ]:
%matplotlib inline
from parcels import *
import numpy as np
import math
from datetime import timedelta

## Running particles in an idealised grid <a name="idealised"></a>

The first step to running particles with PARCELS is to define a `grid` object. In this first case, we use a geostrophic flow of two idealised moving Gaussian eddies. That field is saved in NetCDF format in the directory `examples/MovingEddies_data`. Since we know that the format is in `NEMO` format, we can call these files using the function `Grid.from_nemo`.

In [ ]:
grid = Grid.from_nemo("examples/MovingEddies_data/moving_eddies")

The next step is to define a `ParticleSet`, that works on the grid class. In this case, we start 2 particles on a line between (3.3E, 46N) and (3.3E, 47.8N). Note that we use `JITParticle` as `pclass`, because we will be executing the advection in JIT (Just-In-Time) mode. The alternative is to run in `scipy` mode, in which case pclass is simply `Particle`

In [ ]:
pset = grid.ParticleSet(size=2, pclass=JITParticle, start=(3.3, 46.), finish=(3.3, 47.8))

Print the `ParticleSet` to see where they start

In [ ]:
print pset

This output shows for each particle the (longitude, latitude, depth), and then in square brackets the grid indices of the longitude and latitude

The final step is to execute the `ParticelSet`. We run the particles using the `AdvectionRK4` kernel, which is a 4th order Runge-Kutte implementation. We run the particles for 3 days (using the `timedelta` function from `datetime`), at an RK4 timestep of 5 minutes. We plot the output every hour. We store the trajectory information in a file called `EddyParticles.nc`.

In [ ]:
kernel = AdvectionRK4
pset.execute(kernel, runtime=timedelta(days=3), dt=timedelta(minutes=5), interval=timedelta(hours=1), 
             output_file=pset.ParticleFile(name="EddyParticles"))

The code should have run, which can be confirmed by printing the `ParticleSet` again

In [ ]:
print pset

Or by quickly plotting the output file `EddyParticles.nc`.

In [ ]:
run scripts/plotParticles.py -p EddyParticles.nc

Now one of the neat features of PARCELS is that the particles can be plotted as a movie during execution. To rerun the particles while plotting them on top of the sea surface heigh field (`grid.P`), first reinitialise the `ParticleSet` and then re-execute but changing `show_movie` to `grid.P`

In [ ]:
pset = grid.ParticleSet(size=2, pclass=JITParticle,
                        start=(3.3, 46.), finish=(3.3, 47.8))
pset.execute(kernel, runtime=timedelta(days=3), dt=timedelta(minutes=5), interval=timedelta(hours=1),
             show_movie=grid.U)

## Adding a custom behaviour kernel <a name="customkernel"></a>

A key feature of PARCELS is the ability to quickly create very simple kernels, and add them to the execution. Kernels in this case our little snippets of code that alter the trajectories of the particles. 
In this example, we'll create a simple kernel where particles obtain an extra 5 m/s westward velocity after 1 day. Of course, this is not very realistic scenario, but it nicely illustrates the power of custom kernels.

In [ ]:
def WestVel(particle, grid, time, dt):
    if time > 86400:
        uvel = -2.
        particle.lon += uvel * dt / 1852 / 60

Now reset the `ParticleSet` again, and re-execute. Note that we have now changed `kernel` to be `AdvectionRK4 + k_WestVel`, where k_WestVel is the `WestVel` function defined above cast into a `Kernel` object (via the `pset.Kernel` call). Note also that we also run in `scipy` mode now (we have changed the `pclass` in the `ParticleSet` to `Particle`); in general it is recommended to run in `scipy` mode when debugging code.

In [ ]:
pset = grid.ParticleSet(size=2, pclass=Particle, start=(3.3, 46.), finish=(3.3, 47.8))
k_WestVel = pset.Kernel(WestVel)
kernel = AdvectionRK4 + k_WestVel
pset.execute(kernel, runtime=timedelta(days=3), dt=timedelta(minutes=5), interval=timedelta(hours=1),
            output_file=pset.ParticleFile(name="EddyParticles_WestVel"))

And now plot this new particle field

In [ ]:
run scripts/plotParticles.py -p EddyParticles_WestVel.nc

## Reading in data from NetCDF files <a name="netcdf"></a>

In most cases, you will want to advect particles within pre-computed velocity fields. If these velocity fields are stored in NetCDF format, it is fairly easy to load them into the `Grid.from_netcdf()` function.

The `examples` directory contains a set of [GlobCurrent](http://globcurrent.ifremer.fr/products-data/products-overview) files of the region around South Africa. 

First, set the filenmaes containing the zonal (U) and meridional (V) velocities. You can use wildcards (`*`) and the filenames for U and V can be the same (as in this case)

In [ ]:
filenames = {'U': "examples/GlobCurrent_example_data/20*-GLOBCURRENT-L4-CUReul_hs-ALT_SUM-v02.0-fv01.0.nc",
             'V': "examples/GlobCurrent_example_data/20*-GLOBCURRENT-L4-CUReul_hs-ALT_SUM-v02.0-fv01.0.nc"}

Then, define a dictionary of the variables (`U` and `V`) and dimensions (`lon`, `lat` and `time`; note that GlobCurrent data does not have a `depth` dimension)

In [ ]:
variables = {'U': 'eastward_eulerian_current_velocity', 
             'V': 'northward_eulerian_current_velocity'}
dimensions = {'lat': 'lat', 
              'lon': 'lon', 
              'time': 'time'}

Finally, read in the grid using `filenames`, `variables` and `dimensions`

In [ ]:
grid = Grid.from_netcdf(filenames, variables, dimensions)

Now define a `ParticleSet`, in this case with 1 particle starting at (27E, 35S)

In [ ]:
pset = grid.ParticleSet(2, pclass=JITParticle, lon=[25, 27], lat=[-35, -35])

And finally execute the `ParticleSet` for 1 week using 4th order Runge-Kutta, and show a movie on top of the zonal velocity

In [ ]:
pset.execute(AdvectionRK4, 
             runtime=timedelta(weeks=1), dt=timedelta(minutes=5), interval=timedelta(hours=6), 
             show_movie=grid.U)